<a href="https://colab.research.google.com/github/AusafMo/CustomDocument-ChatBot/blob/main/PineCode_and_Retriever_based_QA_Bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python --force-reinstall --upgrade --no-cache-dir --verbose

# **QA on a .txt File**

In [1]:
!pip install langchain
# !pip install pypdf
!pip install unstructured
!pip install sentence_transformers
!pip install pinecone-client
!pip install llama-cpp-python
!pip install huggingface_hub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 358.9/358.9 kB 30.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.1/275.1 kB 28.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 64.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 39.6 MB/s eta 0:00:00
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993224 sha256=203412e2cebea6547ce563f78f0147a8d975ceb47e6e0f1c084d375a7944b1e8
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
Successfully built langdetect
     ━━━━━━━━━━━━

In [93]:
import pinecone
import os
from langchain.document_loaders import PyPDFLoader, OnlinePDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
from sentence_transformers import SentenceTransformer
from langchain.llms import LlamaCpp
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from huggingface_hub import hf_hub_download
from langchain.chains.question_answering import load_qa_chain
from google.colab import userdata


In [94]:
HUGGINGFACEHUB_API_TOKEN = userdata.get('HUGGINGFACEHUB_API_TOKEN')
PINECONE_API_KEY = userdata.get('PINECONE_API_KEY')
PINECONE_API_ENV = userdata.get('PINECONE_API_ENV')

In [95]:
# pinecone.delete_index("bot")
# pinecone.create_index("bot", dimension=384)
embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
# initialize pinecone
pinecone.init(
    api_key=PINECONE_API_KEY,  # find at app.pinecone.io
    environment=PINECONE_API_ENV  # next to api key in console
)
index_name = "bot" # put in the name of your pinecone index here

In [96]:
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import TextLoader

loader = TextLoader("Idea Description.txt")
data = loader.load()

data
text_splitter=RecursiveCharacterTextSplitter(chunk_size=256, chunk_overlap=32)
docs=text_splitter.split_documents(data)
len(docs)
docs

[Document(page_content='Problem Statement: Identification of Different Medicinal Plants/Raw materials through Image Processing Using Machine Learning Algorithms\n\n\nSolutions :', metadata={'source': 'Idea Description.txt'}),
 Document(page_content='A user-friendly Web-based and Android-based platform for users to upload images, find out whether it’s a medical plant/material or not, and provide detailed descriptions of the benefits of the recognized leaf/material using deep learning techniques.', metadata={'source': 'Idea Description.txt'}),
 Document(page_content='Our Idea/Solution is executed in 4 major phases, viz. Data Collection and Processing, Model Building, Platform Integration and Rigorous Testing. Lets look at them comprehensively :\n\nPhase 1: Dataset Collection', metadata={'source': 'Idea Description.txt'}),
 Document(page_content='We have used a publicly available dataset as a starter for the problem, and plan to make this dataset more robust by adding more data points.\n\

In [142]:
try :
  docsearch = Pinecone.from_existing_index(index_name, embeddings)
  print('existing')
except:
  docsearch = Pinecone.from_texts([t.page_content for t in docs], embeddings, index_name=index_name)
  print('new')

existing


In [98]:
query = 'Frontend'
docs = docsearch.similarity_search(query)
docs

[Document(page_content='Front-End Component:'),
 Document(page_content='The front-end component is the user-facing part of the platform, responsible for interacting with users and facilitating the input of data, particularly images in this context. Users typically access the front end through a user interface, which may be'),
 Document(page_content='Back-End Component:'),
 Document(page_content='3. Interaction between components: The back-end component serves as a bridge between the front-end and the machine learning model. It communicates with the front end to receive user inputs and, once the inference is complete, sends the results back to')]

In [39]:
# Callbacks support token-wise streaming
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])
# Verbose is required to pass to the callback manager
model_name_or_path = "TheBloke/CodeLlama-13B-Python-GGUF"
model_basename = "codellama-13b-python.Q5_K_M.gguf"
model_path = hf_hub_download(repo_id=model_name_or_path, filename=model_basename)

In [41]:
n_gpu_layers = 32
n_batch = 512
llm = LlamaCpp(
              model_path=model_path,
              max_tokens=256,
              n_gpu_layers=n_gpu_layers,
              n_batch=n_batch,
              callback_manager=callback_manager,
              n_ctx=1024,
              verbose=True,
)

AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 0 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 | 


In [42]:
chain=load_qa_chain(llm, chain_type="stuff")

In [43]:
query = 'What is the problem statement'
docs = docsearch.similarity_search(query)
docs

[Document(page_content='Features of our Solution :'),
 Document(page_content='Features of our Solution :'),
 Document(page_content='Features of our Solution :'),
 Document(page_content='Tech Stack Description : \n\tWe have used a diverse tech stack to address the problem statement. Tech-Stack for different components are as f\n\n\tModel Development:')]

In [99]:
from langchain.llms import HuggingFaceHub

In [134]:
llm=HuggingFaceHub(repo_id="google/flan-t5-xxl", model_kwargs={"temperature":1, "max_length":64}, huggingfacehub_api_token = HUGGINGFACEHUB_API_TOKEN)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:127: FutureWarning: '__init__' (from 'huggingface_hub.inference_api') is deprecated and will be removed from version '0.19.0'. `InferenceApi` client is deprecated in favor of the more feature-complete `InferenceClient`. Check out this guide to learn how to convert your script to use it: https://huggingface.co/docs/huggingface_hub/guides/inference#legacy-inferenceapi-client.
  warnings.warn(warning_message, FutureWarning)


In [148]:
chain=load_qa_chain(llm, chain_type="stuff")

In [136]:
query="what are the vaious tech stack used in detail ?"
docs=docsearch.similarity_search(query)
print(docs)

[Document(page_content='Tech Stack Description : \n\tWe have used a diverse tech stack to address the problem statement. Tech-Stack for different components are as f\n\n\tModel Development:'), Document(page_content='Tech Stack Description : \n\tWe have used a diverse tech stack to address the problem statement. Tech-Stack for different components are as f\n\n\tModel Development:'), Document(page_content='Tech Stack Description : \n\tWe have used a diverse tech stack to address the problem statement. Tech-Stack for different components are as f\n\n\tModel Development:'), Document(page_content='for deployment, scaling, and management, ensuring that the platform is robust and responsive.')]


In [138]:
answer = chain.run(input_documents=docs, question=query)
answer

'for deployment, scaling, and management, ensuring that the platform is robust and responsive'

# **RetrieverQA**

In [150]:
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

prompt_template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}
Answer :"""
PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)
chain_type_kwargs = {"prompt": PROMPT}
qa = RetrievalQA.from_chain_type(llm, chain_type="stuff", retriever=docsearch.as_retriever(), chain_type_kwargs=chain_type_kwargs)

In [154]:
query = "Explain the architecture in very detail"
qa.run(query)

'This two-tier architecture neatly separates the user interaction and data collection (front-end)'

# **A (not so) Elegant Function**

In [ ]:
# !pip install pinecone-client
# !pip install huggingface-hub
# !pip install sentence-transformers
# !pip install langchain
# !pip install pypdf
!pip install llama-cpp-python

In [64]:
# Import required libraries
import pinecone
import os
from langchain.document_loaders import PyPDFLoader, OnlinePDFLoader, TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
from sentence_transformers import SentenceTransformer
from langchain.llms import LlamaCpp
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from huggingface_hub import hf_hub_download
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import HuggingFaceHub

from google.colab import userdata

# Initialize Pinecone
HUGGINGFACEHUB_API_TOKEN = userdata.get('HUGGINGFACEHUB_API_TOKEN')
PINECONE_API_KEY = userdata.get('PINECONE_API_KEY')
PINECONE_API_ENV = userdata.get('PINECONE_API_ENV')

pinecone.init(api_key=PINECONE_API_KEY, environment=PINECONE_API_ENV)
index_name = "bot"  # Choose a name for your Pinecone index

# Define functions to clean up the code
def setup_embeddings():
    # Initialize embeddings
    embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

def setup_pdf_loader(file_path):
    # Initialize PDF loader
    return PyPDFLoader(file_path)

def split_text_into_chunks(data, chunk_size=500, chunk_overlap=0):
    # Split text into chunks
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    docs = text_splitter.split_documents(data)
    return docs

def initialize_pinecone_index(embeddings, docs, index_name):
    try:
        # If the index already exists, load it
        docsearch = Pinecone.from_existing_index(index_name, embeddings)
    except:
        # Otherwise, create a new index
        docsearch = Pinecone.from_texts([t.page_content for t in docs], embeddings, index_name=index_name)
    return docsearch

def setup_llama_cpp(model_name_or_path, callback_manager, max_tokens=256, n_gpu_layers=32, n_batch=512, n_ctx=1024):
    # Download and set up LlamaCpp
    model_basename = "codellama-13b-python.Q5_K_M.gguf"
    model_path = hf_hub_download(repo_id=model_name_or_path, filename=model_basename)

    llm = LlamaCpp(
        model_path=model_path,
        max_tokens=max_tokens,
        n_gpu_layers=n_gpu_layers,
        n_batch=n_batch,
        callback_manager=callback_manager,
        n_ctx=n_ctx,
        verbose=True,
    )

    return llm

def load_qa_model(llm, chain_type="stuff"):
    # Load question-answering chain
    return load_qa_chain(llm, chain_type=chain_type)

def query_documents(docsearch, query):
    # Perform a similarity search
    return docsearch.similarity_search(query)

def determine_file_type(file_path):
    if file_path.lower().endswith('.pdf'):
        return 'pdf'
    elif file_path.lower().endswith('.txt'):
        return 'txt'
    else:
        raise ValueError("Unsupported file type. Only PDF and TXT files are supported.")

# Main code
def main():
    # Provide the path to the file (PDF or TXT)
    file_path = "/content/Idea Description.txt"

    # Determine the file type
    file_type = determine_file_type(file_path)

    if file_type == 'pdf':
        # Step 1: Initialize PDF loader
        pdf_loader = setup_pdf_loader(file_path)

        # Step 2: Load the PDF data
        data = pdf_loader.load()

    elif file_type == 'txt':
        # Read the text data from the TXT file
        loader = TextLoader("Idea Description.txt")
        data = loader.load()


    # Step 3: Split the text into chunks
    docs = split_text_into_chunks(data)
    print(docs)

    # Step 4: Initialize embeddings
    embeddings = setup_embeddings()

    # Step 5: Initialize Pinecone index
    docsearch = initialize_pinecone_index(embeddings, docs, index_name)

    # Step 6: Set up LlamaCpp
    callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])
    # llm = setup_llama_cpp(
    #     model_name_or_path="TheBloke/CodeLlama-13B-Python-GGUF",
    #     callback_manager=callback_manager,
    # )

    # using huggingface
    llm=HuggingFaceHub(repo_id="google/flan-t5-xxl", model_kwargs={"temperature":0.5, "max_length":512}, huggingfacehub_api_token = HUGGINGFACEHUB_API_TOKEN)

    # Step 7: Load the question-answering model
    chain = load_qa_model(llm, chain_type="stuff")

    # Step 8: Perform a similarity search
    query = "What are the various technologies used ?"
    docs = query_documents(docsearch, query)

    # Step 9: Run question answering on the retrieved documents
    print(chain.run(input_documents=docs, question=query))


if __name__ == "__main__":
    main()

[Document(page_content='Problem Statement: Identification of Different Medicinal Plants/Raw materials through Image Processing Using Machine Learning Algorithms\n\n\nSolutions :\n\nA user-friendly Web-based and Android-based platform for users to upload images, find out whether it’s a medical plant/material or not, and provide detailed descriptions of the benefits of the recognized leaf/material using deep learning techniques.', metadata={'source': 'Idea Description.txt'}), Document(page_content='Our Idea/Solution is executed in 4 major phases, viz. Data Collection and Processing, Model Building, Platform Integration and Rigorous Testing. Lets look at them comprehensively :\n\nPhase 1: Dataset Collection', metadata={'source': 'Idea Description.txt'}), Document(page_content='We have used a publicly available dataset as a starter for the problem, and plan to make this dataset more robust by adding more data points.\n\t\n\tCurrent Dataset Description:\n\t\tThe dataset consists of images o

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:127: FutureWarning: '__init__' (from 'huggingface_hub.inference_api') is deprecated and will be removed from version '0.19.0'. `InferenceApi` client is deprecated in favor of the more feature-complete `InferenceClient`. Check out this guide to learn how to convert your script to use it: https://huggingface.co/docs/huggingface_hub/guides/inference#legacy-inferenceapi-client.
  warnings.warn(warning_message, FutureWarning)


a user interface, which may be a web application, mobile app, or other user-
